In [1]:
import os
import sys
import torch
import requests
import tarfile
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from skimage import io, transform
from torchvision import transforms, datasets
from torchvision.datasets import VisionDataset
from typing import Any, Callable, Dict, List, Optional, Tuple
from torch.utils.data import Dataset, DataLoader

In [3]:
class NotMNIST(VisionDataset):
    # notMNIST 데이터셋의 다운로드 URL
    resource_url = 'http://yaroslavvb.com/upload/notMNIST/notMNIST_large.tar.gz'

    # 초기화 함수
    def __init__(self, root: str, train: bool = True,
                 transform: Optional[Callable] = None,
                 target_transform: Optional[Callable] = None,
                 download: bool = False) -> None:
        super(NotMNIST, self).__init__(root, transform=transform, target_transform=target_transform)

        # 데이터셋이 존재하지 않으면 다운로드
        if not self._check_exists() or download:
            self.download()

        # 데이터 로드
        self.data, self.targets = self._load_data()

    # 데이터셋의 길이 반환
    def __len__(self):
        return len(self.data)

    # 인덱스에 해당하는 데이터 반환
    def __getitem__(self, index):
        image_name = self.data[index]
        image = io.imread(image_name)
        label = self.targets[index]

        if self.transform:
            image = self.transform(image)

        return image, label

    # 데이터 로드 함수
    def _load_data(self):
        filepath = self.image_folder
        data = []
        targets = []

        # 각 클래스별로 데이터 로드
        for target in os.listdir(filepath):
            filenames = [os.path.abspath(os.path.join(filepath, target, x)) for x in os.listdir(os.path.join(filepath, target))]
            targets.extend([target] * len(filenames))
            data.extend(filenames)

        return data, targets

    # 원본 데이터 폴더 경로
    @property
    def raw_folder(self) -> str:
        return os.path.join(self.root, self.__class__.__name__, 'raw')

    # 이미지 폴더 경로
    @property
    def image_folder(self) -> str:
        return os.path.join(self.root, 'notMNIST_large')

    # 데이터 다운로드 함수
    def download(self) -> None:
        os.makedirs(self.raw_folder, exist_ok=True)
        fname = self.resource_url.split("/")[-1]
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
        filesize = int(requests.head(self.resource_url, headers={"User-Agent": user_agent}).headers["Content-Length"])

        # 데이터 다운로드 진행 상황 표시
        with requests.get(self.resource_url, stream=True, headers={"User-Agent": user_agent}) as r, \
                open(os.path.join(self.raw_folder, fname), "wb") as f, \
                tqdm(unit="B", unit_scale=True, unit_divisor=1024, total=filesize, file=sys.stdout, desc=fname) as progress:
            for chunk in r.iter_content(chunk_size=1024):
                datasize = f.write(chunk)
                progress.update(datasize)

        # 다운로드한 파일 압축 해제
        self._extract_file(os.path.join(self.raw_folder, fname), target_path=self.root)

    # 파일 압축 해제 함수
    def _extract_file(self, fname, target_path) -> None:
        tag = "r:gz" if fname.endswith("tar.gz") else "r:"
        with tarfile.open(fname, tag) as tar:
            tar.extractall(path=target_path)

    # 데이터셋 존재 여부 확인 함수
    def _check_exists(self) -> bool:
        return os.path.exists(self.raw_folder)


In [4]:
# 데이터셋 생성
dataset = NotMNIST("data", download=True)

notMNIST_large.tar.gz: 100%|██████████| 236M/236M [00:31<00:00, 7.80MB/s] 


KeyboardInterrupt: 

In [5]:
# 새로운 그림(figure) 객체를 생성합니다.
fig = plt.figure()

# 8개의 샘플 이미지를 출력하기 위한 반복문입니다.
for i in range(8):
    # i번째 샘플 데이터를 가져옵니다.
    sample = dataset[i]

    # 1행 4열의 서브플롯 중 i+1번째 위치에 그래프를 그립니다.
    ax = plt.subplot(1, 4, i + 1)

    # 레이아웃을 조절하여 그래프 간의 간격을 최적화합니다.
    plt.tight_layout()

    # 서브플롯의 제목을 설정합니다.
    ax.set_title('Sample #{}'.format(i))

    # 서브플롯의 축을 숨깁니다.
    ax.axis('off')

    # 샘플 이미지를 출력합니다.
    plt.imshow(sample[0])

    # 4개의 샘플 이미지를 출력한 후 그림을 화면에 표시하고 반복문을 종료합니다.
    if i == 3:
        plt.show()
        break


NameError: name 'dataset' is not defined

<Figure size 640x480 with 0 Axes>

In [ ]:
# 데이터 전처리를 위한 변환(Transform) 객체를 생성합니다.
# 여러 전처리 단계를 순차적으로 적용하기 위해 Compose를 사용합니다.
data_transform = transforms.Compose([

        # 224x224 크기로 무작위로 이미지를 잘라냅니다.
        transforms.RandomCrop(224),

        # 0.5의 확률로 이미지를 수평으로 뒤집습니다.
        transforms.RandomHorizontalFlip(),

        # 이미지를 텐서(Tensor) 형태로 변환합니다.
        transforms.ToTensor(),

        # 주어진 평균과 표준편차를 사용하여 이미지를 정규화합니다.
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

# NotMNIST 데이터셋을 로드합니다. 다운로드는 하지 않습니다.
dataset = NotMNIST("data", download=False)

In [ ]:
# 데이터셋을 배치 크기로 나누어 로드하기 위한 DataLoader 객체를 생성합니다.
# 배치 크기는 128, 데이터를 섞어서 로드합니다.
dataset_loader = torch.utils.data.DataLoader(dataset,
                                             batch_size=128, shuffle=True)

In [ ]:
# DataLoader에서 첫 번째 배치의 특성(features)과 레이블(labels)을 가져옵니다.
train_features, train_labels = next(iter(dataset_loader))

In [ ]:
# 첫 번째 배치의 특성의 형태(shape)를 출력합니다.
train_features.shape

In [ ]:
# 첫 번째 배치의 레이블을 출력합니다.
train_labels